In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
context = """
🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch, and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "Which deep learning libraries back 🤗 Transformers?"

In [2]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [3]:
inputs = tokenizer(question, context, return_tensors="pt")

In [4]:
inputs


{'input_ids': tensor([[  101,  5979,  1996,  3776,  9818,  1171,   100, 25267,   136,   102,
           100, 25267,  1110,  5534,  1118,  1103,  1210,  1211,  1927,  1996,
          3776,  9818,   783, 13612,   117,   153,  1183,  1942,  1766,  1732,
           117,  1105,  5157, 21484,  2271,  6737,   783,  1114,   170,  2343,
          1306,  2008,  9111,  1206,  1172,   119,  1135,   112,   188, 21546,
          1106,  2669,  1240,  3584,  1114,  1141,  1196, 10745,  1172,  1111,
          1107, 16792,  1114,  1103,  1168,   119,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [6]:
len(inputs.sequence_ids())

67

In [8]:
inputs.input_ids.shape

torch.Size([1, 67])

In [10]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

2025-09-18 00:27:18.734542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758155239.125283      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758155239.228689      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [12]:
outputs = model(**inputs)

In [17]:
start_logits, end_logits = outputs.start_logits, outputs.end_logits

In [18]:
start_logits.shape, end_logits.shape

(torch.Size([1, 67]), torch.Size([1, 67]))

In [20]:
import torch
sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
mask = torch.tensor(mask)[None]

start_logits[mask] = -10000
end_logits[mask] = -10000

In [29]:
start_prob = torch.nn.functional.softmax(start_logits, dim=-1)[-1]
end_prob = torch.nn.functional.softmax(end_logits, dim=-1)[-1]



In [31]:
scores = start_prob[:, None] * end_prob[None, :]


In [35]:
scores = torch.triu(scores)

In [43]:
scores.argmax()

tensor(1576)

In [45]:
torch.flatten(scores)[1576]

tensor(0.9803, grad_fn=<SelectBackward0>)

In [36]:
max_index = scores.argmax().item()

In [37]:
max_index

1576

In [47]:
start_index = max_index // scores.shape[1]

In [48]:
end_index= max_index % scores.shape[1]

In [49]:
inputs_with_offsets = tokenizer(question, context, return_offsets_mapping=True)
offsets = inputs_with_offsets["offset_mapping"]

start_char, _ = offsets[start_index]
_, end_char = offsets[end_index]
answer = context[start_char:end_char]

In [50]:
answer

'Jax, PyTorch, and TensorFlow'

In [51]:
sentence = "This sentence is not too long but we are going to split it anyway."
inputs = tokenizer(
    sentence, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)


In [59]:
inputs

{'input_ids': [[101, 1188, 5650, 1110, 1136, 102], [101, 1110, 1136, 1315, 1263, 102], [101, 1315, 1263, 1133, 1195, 102], [101, 1133, 1195, 1132, 1280, 102], [101, 1132, 1280, 1106, 3325, 102], [101, 1106, 3325, 1122, 4050, 102], [101, 1122, 4050, 119, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 0, 0, 0, 0, 0, 0]}

In [58]:
1+1

2

In [60]:
for i in inputs["input_ids"]:
    print(tokenizer.decode(i))

[CLS] This sentence is not [SEP]
[CLS] is not too long [SEP]
[CLS] too long but we [SEP]
[CLS] but we are going [SEP]
[CLS] are going to split [SEP]
[CLS] to split it anyway [SEP]
[CLS] it anyway. [SEP]


In [61]:
long_context = """
🤗 Transformers: State of the Art NLP

🤗 Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction,
question answering, summarization, translation, text generation and more in over 100 languages.
Its aim is to make cutting-edge NLP easier to use for everyone.

🤗 Transformers provides APIs to quickly download and use those pretrained models on a given text, fine-tune them on your own datasets and
then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and
can be modified to enable quick research experiments.

Why should I use transformers?

1. Easy-to-use state-of-the-art models:
  - High performance on NLU and NLG tasks.
  - Low barrier to entry for educators and practitioners.
  - Few user-facing abstractions with just three classes to learn.
  - A unified API for using all our pretrained models.
  - Lower compute costs, smaller carbon footprint:

2. Researchers can share trained models instead of always retraining.
  - Practitioners can reduce compute time and production costs.
  - Dozens of architectures with over 10,000 pretrained models, some in more than 100 languages.

3. Choose the right framework for every part of a model's lifetime:
  - Train state-of-the-art models in 3 lines of code.
  - Move a single model between TF2.0/PyTorch frameworks at will.
  - Seamlessly pick the right framework for training, evaluation and production.

4. Easily customize a model or an example to your needs:
  - We provide examples for each architecture to reproduce the results published by its original authors.
  - Model internals are exposed as consistently as possible.
  - Model files can be used independently of the library for quick experiments.

🤗 Transformers is backed by the three most popular deep learning libraries — Jax, PyTorch and TensorFlow — with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""


In [62]:
inputs = tokenizer(question, long_context, max_length=384, truncation="only_second")
print(tokenizer.decode(inputs["input_ids"]))

[CLS] Which deep learning libraries back [UNK] Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified A

In [63]:
inputs

{'input_ids': [101, 5979, 1996, 3776, 9818, 1171, 100, 25267, 136, 102, 100, 25267, 131, 1426, 1104, 1103, 2051, 21239, 2101, 100, 25267, 2790, 4674, 1104, 3073, 4487, 9044, 3584, 1106, 3870, 8249, 1113, 6685, 1216, 1112, 5393, 117, 1869, 16026, 117, 2304, 10937, 117, 7584, 7317, 2734, 117, 5179, 117, 3087, 3964, 1105, 1167, 1107, 1166, 1620, 3483, 119, 2098, 6457, 1110, 1106, 1294, 5910, 118, 2652, 21239, 2101, 5477, 1106, 1329, 1111, 2490, 119, 100, 25267, 2790, 20480, 1116, 1106, 1976, 9133, 1105, 1329, 1343, 3073, 4487, 9044, 3584, 1113, 170, 1549, 3087, 117, 2503, 118, 9253, 1172, 1113, 1240, 1319, 2233, 27948, 1105, 1173, 2934, 1172, 1114, 1103, 1661, 1113, 1412, 2235, 10960, 119, 1335, 1103, 1269, 1159, 117, 1296, 185, 25669, 8613, 13196, 13682, 1126, 4220, 1110, 3106, 2484, 20717, 1673, 1105, 1169, 1129, 5847, 1106, 9396, 3613, 1844, 7857, 119, 2009, 1431, 146, 1329, 11303, 1468, 136, 122, 119, 12167, 118, 1106, 118, 1329, 1352, 118, 1104, 118, 1103, 118, 1893, 3584, 131, 118, 

In [64]:
sentences = [
    "This sentence is not too long but we are going to split it anyway.",
    "This sentence is shorter but will still get split.",
]
inputs = tokenizer(
    sentences, truncation=True, return_overflowing_tokens=True, max_length=6, stride=2
)

print(inputs["overflow_to_sample_mapping"])

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]


In [65]:
inputs

{'input_ids': [[101, 1188, 5650, 1110, 1136, 102], [101, 1110, 1136, 1315, 1263, 102], [101, 1315, 1263, 1133, 1195, 102], [101, 1133, 1195, 1132, 1280, 102], [101, 1132, 1280, 1106, 3325, 102], [101, 1106, 3325, 1122, 4050, 102], [101, 1122, 4050, 119, 102], [101, 1188, 5650, 1110, 7681, 102], [101, 1110, 7681, 1133, 1209, 102], [101, 1133, 1209, 1253, 1243, 102], [101, 1253, 1243, 3325, 119, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]}

In [66]:
inputs = tokenizer(
    question,
    long_context,
    stride=128,
    max_length=384,
    padding="longest",
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

In [67]:
_ = inputs.pop("overflow_to_sample_mapping")
offsets = inputs.pop("offset_mapping")

inputs = inputs.convert_to_tensors("pt")
print(inputs["input_ids"].shape)

torch.Size([2, 384])


In [69]:
for id in inputs['input_ids']:
    print(tokenizer.decode(id))

[CLS] Which deep learning libraries back [UNK] Transformers? [SEP] [UNK] Transformers : State of the Art NLP [UNK] Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation and more in over 100 languages. Its aim is to make cutting - edge NLP easier to use for everyone. [UNK] Transformers provides APIs to quickly download and use those pretrained models on a given text, fine - tune them on your own datasets and then share them with the community on our model hub. At the same time, each python module defining an architecture is fully standalone and can be modified to enable quick research experiments. Why should I use transformers? 1. Easy - to - use state - of - the - art models : - High performance on NLU and NLG tasks. - Low barrier to entry for educators and practitioners. - Few user - facing abstractions with just three classes to learn. - A unified A

In [70]:
outputs = model(**inputs)

start_logits = outputs.start_logits
end_logits = outputs.end_logits
print(start_logits.shape, end_logits.shape)

torch.Size([2, 384]) torch.Size([2, 384])


In [71]:
sequence_ids = inputs.sequence_ids()
# Mask everything apart from the tokens of the context
mask = [i != 1 for i in sequence_ids]
# Unmask the [CLS] token
mask[0] = False
# Mask all the [PAD] tokens
mask = torch.logical_or(torch.tensor(mask)[None], (inputs["attention_mask"] == 0))

start_logits[mask] = -10000
end_logits[mask] = -10000

In [72]:
start_probabilities = torch.nn.functional.softmax(start_logits, dim=-1)
end_probabilities = torch.nn.functional.softmax(end_logits, dim=-1)

In [73]:
candidates = []
for start_probs, end_probs in zip(start_probabilities, end_probabilities):
    scores = start_probs[:, None] * end_probs[None, :]
    idx = torch.triu(scores).argmax().item()

    start_idx = idx // scores.shape[1]
    end_idx = idx % scores.shape[1]
    score = scores[start_idx, end_idx].item()
    candidates.append((start_idx, end_idx, score))
print(candidates)

[(0, 18, 0.3386705815792084), (173, 184, 0.9714869856834412)]


In [74]:
for candidate, offset in zip(candidates, offsets):
    start_token, end_token, score = candidate
    start_char, _ = offset[start_token]
    _, end_char = offset[end_token]
    answer = long_context[start_char:end_char]
    result = {"answer": answer, "start": start_char, "end": end_char, "score": score}
    print(result)

{'answer': '\n🤗 Transformers: State of the Art NLP', 'start': 0, 'end': 37, 'score': 0.3386705815792084}
{'answer': 'Jax, PyTorch and TensorFlow', 'start': 1892, 'end': 1919, 'score': 0.9714869856834412}
